In [37]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import torch
import os
import re
from PyPDF2 import PdfReader
from transformers import AutoModel, AutoTokenizer

In [2]:
# model_name = 'DeepPavlov/rubert-base-cased-sentence'
model_name = 'DeepPavlov/bert-base-multilingual-cased-sentence'
# model_name = 'DeepPavlov/distilrubert-small-cased-conversational'
# model_name = 'DeepPavlov/distilrubert-tiny-cased-conversational'
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [3]:
def get_first_header(note):
    if '## ' in note:
        first_header_prefix = '## '
        if '### ' in note:
            first_header_prefix = '### '

        first_header = note.split(first_header_prefix)[1].split('\n')[0]
    else:
        first_header = ''

    return first_header

def clean(note):
    # remove zero-links
    note = re.sub(r'\[.*\]', '', note)

    # remove tags and headers
    note = re.sub(r'\#.*\n', '', note)

    # remove \n
    note = re.sub('\n', ' ', note)

    # remove lines
    note = re.sub('---', ' ', note)

    # remove **
    note = re.sub('\*', '', note)
    
    return note

def num_letters(note):
    return len(re.sub(r'[^а-яА-Яa-zA-Z]', '', note))

In [4]:
length_thr = 100
device = torch.device(0)
# device = 'cpu'
encode_kwargs = {'truncation': True, 'padding': 'max_length', 'pad_to_multiple_of': 1, 'max_length':512}

In [56]:
def get_abstract(text):
    if ('abstract' in text.lower()) and ('ntroduction' in text.lower()):
        start_pos = text.lower().index('abstract') + len('abstract')
        end_pos = text.lower().index('ntroduction') - 1
        abstract = text[start_pos: end_pos]
    else:
        abstract = text

    abstract = re.sub('-\n', '', abstract)
    abstract = re.sub('\n', ' ', abstract)
    return abstract

In [59]:
files_

['Martins et al. - 2022 - $infty$-former Infinite Memory Transformer.pdf',
 '.zotero-ft-cache',
 '.zotero-ft-info']

In [101]:
db_path = '/home/booydar/Documents/Sync/zotero/storage/'
path = db_path
def parse_storage(path):

    db_df = pd.DataFrame()
    path, folders, files = next(os.walk(path))
    for f in folders:
        f_path = os.path.join(path, f)
        path_, folders_, files_ = next(os.walk(f_path))
        pdfs = [n for n in files_ if '.pdf' in n]
        if not pdfs:
            continue
        paper = pdfs[0]
        paper_path = os.path.join(f_path, paper)
        reader = PdfReader(paper_path)
        page = reader.pages[0]
        first_page = page.extract_text()
        abstract = get_abstract(first_page)

        paper_dict = {'path': paper_path,
                    'paper': paper,
                    'name': re.sub(r'.*-* - ', '', paper)[:-4], 
                    'abstract': abstract}

        db_df = pd.concat([db_df, pd.DataFrame(paper_dict, index=[0])])
        
    return db_df

In [102]:
db_df = parse_storage(db_path)

In [106]:
db_df

,path,paper,name,abstract
0,/home/booydar/Documents/Sync/zotero/storage/SF...,Jaegle et al. - 2022 - Perceiver IO A General ...,Perceiver IO A General Architecture for Structure,A central goal of machine learning is the dev...
0,/home/booydar/Documents/Sync/zotero/storage/KS...,Liška et al. - 2018 - Memorize or generalize S...,Memorize or generalize Searching for a compositio,Neural networks are very powerful learning sy...
0,/home/booydar/Documents/Sync/zotero/storage/TQ...,Yao et al. - 2019 - KG-BERT BERT for Knowledge...,KG-BERT BERT for Knowledge Graph Completion,Knowledge graphs are important resources for ...
0,/home/booydar/Documents/Sync/zotero/storage/EV...,Lample et al. - 2019 - Large Memory Layers wit...,Large Memory Layers with Product Keys,This paper introduces a structured memory whi...
0,/home/booydar/Documents/Sync/zotero/storage/EH...,Voita et al. - 2019 - When a Good Translation ...,When a Good Translation is Wrong in Context Conte,Though machine translation errors caused by t...
...,...,...,...,...
0,/home/booydar/Documents/Sync/zotero/storage/6P...,Sun et al. - 2021 - Do Long-Range Language Mod...,Do Long-Range Language Models Actually Use Lon...,Language models are generally trained on shor...
0,/home/booydar/Documents/Sync/zotero/storage/3G...,Hutchins et al. - 2022 - Block-Recurrent Trans...,Block-Recurrent Transformers,"We introduce the Block-Recurrent Transformer,..."
0,/home/booydar/Documents/Sync/zotero/storage/MF...,Zhang et al. - 2020 - PEGASUS Pre-training wit...,PEGASUS Pre-training with Extracted Gap-sentences,ive Summarization Jingqing Zhang* 1Yao Zhao* 2...
0,/home/booydar/Documents/Sync/zotero/storage/UL...,Ba et al. - 2016 - Using Fast Weights to Atten...,Using Fast Weights to Attend to the Recent Past,"Until recently, research on artiﬁcial neural ..."


In [108]:
out_path = '/home/booydar/Documents/Sync/scientific_graph'

for i, row in db_df.iterrows():
    
    note = ''
    note += '#paper\n---\n\n'
    note += f"##{row['name']}\n\n"
    note += f'{row.abstract}\n\n---\n'

    note_name = row.paper[:-4] + '.md'
    op = os.path.join(out_path, note_name)

    with open(op, 'w') as f:
        f.write(note)

SyntaxError: EOL while scanning string literal (790908752.py, line 7)